<a href="https://colab.research.google.com/github/camillaysm/data-panel/blob/main/Data_Panel_ASEAN_Plus_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Is there an influence of Regulatory Quality, Government Effectiveness, Size of a Country, and Foreign Direct Investment (FDI) on Trade Openness in ASEAN Plus Three countries for the 2003–2022 period ?**

In [ ]:
# Mounting g-drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Read csv
data = pd.read_csv('/content/drive/MyDrive/Dataset /DATA PANEL ASEAN PLUS 3.csv')

In [ ]:
data.head()

,Negara,Tahun,RQ,GE,Size,FDI,TO
0,Indonesia,2003,-0.866097,-0.596455,223080121,-0.254256,53.616494
1,Indonesia,2004,-0.735824,-0.490505,225938595,0.738244,59.761295
2,Indonesia,2005,-0.646948,-0.502645,228805144,2.916115,63.987936
3,Indonesia,2006,-0.396196,-0.348612,231797427,1.347943,56.657127
4,Indonesia,2007,-0.388592,-0.307464,234858289,1.603011,54.829250


Populasi tiap negara yang digambarkan pada kolom 'Size' memiliki satuan Juta Jiwa yang jauh berbeda dari nilai pada kolom lainnya, sehingga dilakukan **transformasi log** untuk mendapat nilai yang lebih selaras dengan kolom/variabel lainnya.

In [ ]:
# Melakukan transformasi ln pada Size supaya data memiliki nilai yang selaras
data['ln_Size'] = np.log(data['Size'])

In [ ]:
# Menghapus data Size
data = data.drop('Size', axis=1)
# Mengganti nama kolom ln_size menjadi Size
data = data.rename(columns={'ln_Size': 'Size'})

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Negara  260 non-null    object 
 1   Tahun   260 non-null    int64  
 2   RQ      260 non-null    float64
 3   GE      260 non-null    float64
 4   FDI     260 non-null    float64
 5   TO      260 non-null    float64
 6   Size    260 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 14.3+ KB


In [ ]:
data.describe()

,Tahun,RQ,GE,FDI,TO,Size
count,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000
mean,2012.500000,0.124180,0.314725,4.702155,104.355903,17.463636
std,5.777402,0.973552,0.996845,5.956551,89.507450,1.939392
min,2003.000000,-2.348573,-1.684041,-1.752934,0.000000,12.777179
25%,2007.750000,-0.509255,-0.349375,1.342135,46.733600,16.446410
50%,2012.500000,0.028859,0.197810,2.848698,87.446456,17.761329
75%,2017.250000,0.912260,1.041648,4.714723,130.861140,18.490143
max,2022.000000,2.252235,2.469660,31.620740,437.326715,21.068528


In [ ]:
# Deskriptif statistik berdasarkan tiap negara kecuali pada kolom tahun
data.groupby('Negara').describe().drop('Tahun', axis=1)

RQ                                                    \
                  count      mean       std       min       25%       50%   
Negara                                                                      
Brunei Darussalam  20.0  0.938123  0.183860  0.580996  0.823671  0.983331   
China              20.0 -0.289564  0.067459 -0.418418 -0.331767 -0.301207   
Filipina           20.0 -0.038972  0.141500 -0.369468 -0.135438  0.025957   
Indonesia          20.0 -0.209233  0.313524 -0.866097 -0.382793 -0.190040   
Jepang             20.0  1.217236  0.129852  1.016683  1.124122  1.170810   
Kamboja            20.0 -0.523858  0.099684 -0.709801 -0.596187 -0.508558   
Korea Selatan      20.0  0.956673  0.148651  0.717727  0.838248  0.985453   
Laos               20.0 -0.995358  0.205120 -1.353559 -1.148840 -0.943878   
Malaysia           20.0  0.573824  0.131564  0.269759  0.528483  0.583864   
Myanmar            20.0 -1.597171  0.622890 -2.348573 -2.198547 -1.667350   
Singapura          20.0  1.983339  0.199626  1.746807  1.783507  1.959005   
Thailand           20.0  0.133913  0.119482 -0.044097  0.063198  0.117011   
Vietnam            20.0 -0.534611  0.121034 -0.676630 -0.630987 -0.586907   

                                         GE            ...          TO  \
                        75%       max count      mean  ...         75%   
Negara                                                 ...               
Brunei Darussalam  1.088384  1.196243  20.0  0.977419  ...  108.525403   
China             -0.232169 -0.179301  20.0  0.239268  ...   53.256250   
Filipina           0.066415  0.127121  20.0  0.054342  ...   72.733508   
Indonesia          0.000730  0.287561  20.0 -0.154152  ...   53.919683   
Jepang             1.337443  1.438773  20.0  1.518178  ...   35.269166   
Kamboja           -0.460470 -0.352435  20.0 -0.781971  ...  130.864930   
Korea Selatan      1.093882  1.145223  20.0  1.086983  ...   92.428785   
Laos              -0.838598 -0.751459  20.0 -0.780248  ...   84.989834   
Malaysia           0.664650  0.799231  20.0  1.005994  ...  180.617620   
Myanmar           -1.057053 -0.621483  20.0 -1.391728  ...    0.000000   
Singapura          2.179307  2.252235  20.0  2.211566  ...  382.896092   
Thailand           0.180080  0.453963  20.0  0.224850  ...  133.164023   
Vietnam           -0.432676 -0.236916  20.0 -0.119081  ...  161.546528   

                               Size                                  \
                          max count       mean       std        min   
Negara                                                                
Brunei Darussalam  147.123112  20.0  12.912295  0.074195  12.777179   
China               64.479191  20.0  21.028633  0.031274  20.976667   
Filipina            87.574644  20.0  18.405946  0.103338  18.233662   
Indonesia           63.987936  20.0  19.338282  0.067782  19.223042   
Jepang              46.843732  20.0  18.661711  0.006479  18.644824   
Kamboja            144.614454  20.0  16.510613  0.084331  16.363995   
Korea Selatan      105.566314  20.0  17.729894  0.027894  17.684466   
Laos                99.059738  20.0  15.694951  0.087401  15.554056   
Malaysia           210.374266  20.0  17.202604  0.097915  17.023908   
Myanmar              0.000000  20.0  17.736682  0.045018  17.664046   
Singapura          437.326715  20.0  15.445597  0.112113  15.230107   
Thailand           140.437012  20.0  18.048293  0.033078  17.986460   
Vietnam            186.675833  20.0  18.311769  0.059503  18.215817   

                                                               
                         25%        50%        75%        max  
Negara                                                         
Brunei Darussalam  12.856064  12.921862  12.974494  13.014783  
China              21.003137  21.029800  21.058200  21.068528  
Filipina           18.324463  18.409243  18.490143  18.565292  
Indonesia          19.284259  19.343926  19.395763  19.434103  
Jepang            

#UNIT ROOT TEST

In [ ]:
!pip install linearmodels --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00


In [ ]:
from statsmodels.tsa.stattools import adfuller

# Uji stasioner untuk variabel
def adf_test(series, name):
    result = adfuller(series, autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', 'Lags Used', 'Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)

    # Interpretasi hasil ADF
    if result[1] <= 0.05:
        print(f"'{name}' stasioner")
    else:
        print(f"'{name}' tidak stasioner")

    print(out)

# Uji ADF untuk setiap variabel
for col in ['RQ', 'GE', 'FDI', 'Size', 'TO']:
    adf_test(data[col], col)

'RQ' stasioner
ADF Test Statistic              -3.258367
p-value                          0.016845
Lags Used                        0.000000
Number of Observations Used    259.000000
dtype: float64
'GE' tidak stasioner
ADF Test Statistic              -2.846328
p-value                          0.051955
Lags Used                        0.000000
Number of Observations Used    259.000000
dtype: float64
'FDI' stasioner
ADF Test Statistic              -3.005331
p-value                          0.034400
Lags Used                       16.000000
Number of Observations Used    243.000000
dtype: float64
'Size' stasioner
ADF Test Statistic              -2.885470
p-value                          0.047064
Lags Used                        0.000000
Number of Observations Used    259.000000
dtype: float64
'TO' stasioner
ADF Test Statistic              -3.350232
p-value                          0.012777
Lags Used                        0.000000
Number of Observations Used    259.000000
dtype: float64


variabel 'GE' tidak stasioner, maka dilakukan diferensiasi

In [ ]:
data['d_GE'] = data['GE'].diff()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Negara  260 non-null    object 
 1   Tahun   260 non-null    int64  
 2   RQ      260 non-null    float64
 3   GE      260 non-null    float64
 4   FDI     260 non-null    float64
 5   TO      260 non-null    float64
 6   Size    260 non-null    float64
 7   d_GE    259 non-null    float64
dtypes: float64(6), int64(1), object(1)
memory usage: 16.4+ KB


In [ ]:
data.isna().sum()

,0
Negara,0
Tahun,0
RQ,0
GE,0
FDI,0
TO,0
Size,0
d_GE,1


In [ ]:
# Fill NaN pada kolom d_RQ
data['d_GE'] = data['d_GE'].fillna(0)

In [ ]:
# Uji ADF untuk setiap variabel
for col in ['RQ', 'd_GE', 'FDI', 'Size', 'TO']:
    adf_test(data[col], col)

'RQ' stasioner
ADF Test Statistic              -3.258367
p-value                          0.016845
Lags Used                        0.000000
Number of Observations Used    259.000000
dtype: float64
'd_GE' stasioner
ADF Test Statistic            -1.574449e+01
p-value                        1.244446e-28
Lags Used                      0.000000e+00
Number of Observations Used    2.590000e+02
dtype: float64
'FDI' stasioner
ADF Test Statistic              -3.005331
p-value                          0.034400
Lags Used                       16.000000
Number of Observations Used    243.000000
dtype: float64
'Size' stasioner
ADF Test Statistic              -2.885470
p-value                          0.047064
Lags Used                        0.000000
Number of Observations Used    259.000000
dtype: float64
'TO' stasioner
ADF Test Statistic              -3.350232
p-value                          0.012777
Lags Used                        0.000000
Number of Observations Used    259.000000
dtype: float

#PEMILIHAN ESTIMASI MODEL

In [ ]:
# Tahun dan Negara adalah index
data = data.set_index(['Negara', 'Tahun'])

In [ ]:
from linearmodels.panel import compare
from linearmodels.panel import PooledOLS, PanelOLS

# Estimasi model Pooled OLS
X_pooled = sm.add_constant(data[['RQ', 'd_GE', 'FDI', 'Size']])
pooled_model = PooledOLS(data['TO'], X_pooled)
pooled_res = pooled_model.fit()

# Estimasi model Fixed Effects
fe_model = PanelOLS(data['TO'], X_pooled, entity_effects=True)
fe_res = fe_model.fit()

# Bandingkan kedua model (Uji Chow)
comparison = compare({'Pooled OLS': pooled_res, 'Fixed Effects': fe_res})
print(comparison)

                   Model Comparison                  
                            Pooled OLS  Fixed Effects
-----------------------------------------------------
Dep. Variable                       TO             TO
Estimator                    PooledOLS       PanelOLS
No. Observations                   260            260
Cov. Est.                   Unadjusted     Unadjusted
R-squared                       0.6995         0.2368
R-Squared (Within)             -1.9755         0.2368
R-Squared (Between)             0.8369        -6.6349
R-Squared (Overall)             0.6995        -6.2992
F-statistic                     148.41         18.846
P-value (F-stat)                0.0000         0.0000
=====================     ============   ============
const                           133.00         2476.4
                              (4.2812)       (7.8342)
RQ                              31.392         5.6813
                              (9.1610)       (1.0482)
d_GE                        

p-value < 0.05 maka FEM terpilih

In [ ]:
from linearmodels.panel import RandomEffects

# Estimasi model Random Effects
re_model = RandomEffects(data['TO'], X_pooled)
re_res = re_model.fit()

# Bandingkan hasil Fixed Effects dan Random Effects (Uji Hausman)
comparison = compare({'Fixed Effects': fe_res, 'Random Effects': re_res})
print(comparison)

                    Model Comparison                    
                         Fixed Effects    Random Effects
--------------------------------------------------------
Dep. Variable                       TO                TO
Estimator                     PanelOLS     RandomEffects
No. Observations                   260               260
Cov. Est.                   Unadjusted        Unadjusted
R-squared                       0.2368            0.1065
R-Squared (Within)              0.2368            0.1267
R-Squared (Between)            -6.6349           -0.0050
R-Squared (Overall)            -6.2992            0.0015
F-statistic                     18.846            7.6024
P-value (F-stat)                0.0000            0.0000
=====================     ============   ===============
const                           2476.4            669.30
                              (7.8342)          (5.9603)
RQ                              5.6813           -1.2333
                              (

FEM Terpilih

In [ ]:
# Menampilkan hasil regresi model terpilih Fixed Effect
fe_model = PanelOLS(data['TO'], X_pooled, entity_effects=True)
fe_res = fe_model.fit()

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                     TO   R-squared:                        0.2368
Estimator:                   PanelOLS   R-squared (Between):             -6.6349
No. Observations:                 260   R-squared (Within):               0.2368
Date:                Fri, Sep 13 2024   R-squared (Overall):             -6.2992
Time:                        14:48:52   Log-likelihood                   -1109.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      18.846
Entities:                          13   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(4,243)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             18.846
                            

#UJI ASUMSI KLASIK

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Uji Multikolinieritas VIF
# Hitung VIF untuk setiap variabel independen
X = data[['RQ', 'd_GE', 'FDI', 'Size']]
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

print(vif_data)

  feature       VIF
0      RQ  1.165537
1    d_GE  1.040105
2     FDI  1.710551
3    Size  1.548398


nilai VIF < 10 maka data tidak terjadi multikolinieritas

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan

# Uji Heteroskedastisitas Breusch-Pagan test
bp_test = het_breuschpagan(fe_res.resids, X_pooled)
print(f"Breusch-Pagan Test Statistic: {bp_test[0]}")
print(f"p-value: {bp_test[1]}")

Breusch-Pagan Test Statistic: 29.036967122501732
p-value: 7.683374187552684e-06


niali pvalue 6.452e-07 < 0.05 maka terjadi heteroskedastisitas

In [ ]:
from statsmodels.stats.stattools import durbin_watson

# Uji Durbin-Watson
dw_stat = durbin_watson(fe_res.resids)
print(f"Durbin-Watson Statistic: {dw_stat}")

Durbin-Watson Statistic: 0.5936979063140053


Ketentuan Durbin Watson:
Mendekati 0 dan 4 ada autokorelasi
Mendekati 2 tidak ada autokorelasi

Hasil:
nilai DW mendekati nol, ada autokorelasi


## Mengatasi heteroskedastisitas dan autokorelasi

Menggunakan Generalized Least Squares (GLS) untuk menghandle heteroskedastisitas dan autokorelasi secara langsung.

In [ ]:
GLS_model = sm.GLS(data['TO'], X_pooled)  # Membuat model GLS tanpa mempertimbangkan autokorelasi

results = GLS_model.fit()
print(results.summary())

                            GLS Regression Results                            
Dep. Variable:                     TO   R-squared:                       0.700
Model:                            GLS   Adj. R-squared:                  0.695
Method:                 Least Squares   F-statistic:                     148.4
Date:                Fri, 13 Sep 2024   Prob (F-statistic):           2.38e-65
Time:                        14:48:53   Log-Likelihood:                -1380.6
No. Observations:                 260   AIC:                             2771.
Df Residuals:                     255   BIC:                             2789.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        132.9958     31.065      4.281      0.0

Nilai DW masih jauh dibawah 2. Karena analisis ini dirasa tidak memerlukan pemenuhan asumsi autokorelasi (tidak apa terjadi persamaan nilai di tahun yang berbeda) maka diputuskan untuk menggunakan Newey-West HAC yang lebih cocok untuk pengabaian autokorelasi.

Note:
GLS dimaksudkan untuk memperbaiki asumsi autokorelasi, dan bisa dilanjutkan metode GLS dengan autoregressive AR(1) untuk mengatasi autokorelasi lebih lanjut.

In [ ]:
OLS_model = sm.OLS(data['TO'], X_pooled)  # Membuat model OLS
results = OLS_model.fit(cov_type='HAC', cov_kwds={'maxlags':2})
          # Covariance Type ('HAC') mengindikasikan bahwa matrix dihitung menggunakan Newey-West Estimator, yang konsisten terhadap heteroskedastisitas dan autokorelasi.
          # maxlags: jumlah lag yang digunakan untuk memperhitungkan autokorelasi. Semakin besar lag, semakin banyak autokorelasi yang akan diakomodasi.

# Output hasil regresi dengan robust standard errors
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     TO   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.695
Method:                 Least Squares   F-statistic:                     40.32
Date:                Fri, 13 Sep 2024   Prob (F-statistic):           3.68e-26
Time:                        14:48:53   Log-Likelihood:                -1380.6
No. Observations:                 260   AIC:                             2771.
Df Residuals:                     255   BIC:                             2789.
Df Model:                           4                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        132.9958     34.711      3.832      0.0

Kualitas regulasi berpengaruh positif signifikan terhadap
keterbukaan perdagangan di negara ASEAN Plus Three periode 2003-2022.

Efektivitas pemerintahan berpengaruh positif tidak signifikan terhadap keterbukaan perdagangan di negara ASEAN Plus Three periode 2003-2022.

FDI berpengaruh positif signifikan terhadap keterbukaan perdagangan di negara ASEAN Plus Three periode 2003-2022.

Ukuran sebuah Negara berpengaruh negatif signifikan terhadap keterbukaan perdagangan di negara ASEAN Plus Three periode 2003-2022.



In [ ]:
from google.colab import auth
auth.authenticate_user()
!git config --global user.email "camillayssam@gmail.com"
!git config --global user.name "camillaysm"
!git clone https://github.com/camillaysm/data-panel.git

Cloning into 'data-panel'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
